<a href="https://colab.research.google.com/github/BRAHIMLOUARDI/movie-web-app/blob/main/Welcome_To_Colaboratory2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import string
import numpy as np

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import LSTM, Input, TimeDistributed, Dense, Activation, RepeatVector, Embedding
from tensorflow.keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

# Path to translation file
path_to_data = '/content/fra.txt'

# Read file
translation_file = open(path_to_data,"r", encoding='utf-8') 
raw_data = translation_file.read()
translation_file.close()

# Parse data
raw_data = raw_data.split('\n')
pairs = [sentence.split('\t') for sentence in  raw_data]
pairs=pairs[0:60000]



In [2]:
def clean_sentence(sentence):
    # Lower case the sentence
    lower_case_sent = sentence.lower()
    # Strip punctuation
    string_punctuation = "?" + "¡" + '¿'+"."+"!" ;
    clean_sentence = lower_case_sent.translate(str.maketrans('', '', string_punctuation))
   
    return clean_sentence

In [3]:
def tokenize(sentences):
    # Create tokenizer
    text_tokenizer = Tokenizer()
    # Fit texts
    text_tokenizer.fit_on_texts(sentences)
    return text_tokenizer.texts_to_sequences(sentences), text_tokenizer

In [6]:

english_sentences = [clean_sentence(pair[0]) for pair in pairs]
french_sentences = [clean_sentence(pair[1]) for pair in pairs]

# Tokenize words
fra_text_tokenized, fra_text_tokenizer = tokenize(french_sentences)
eng_text_tokenized, eng_text_tokenizer = tokenize(english_sentences)

print('Maximum length french sentence: {}'.format(len(max(fra_text_tokenized,key=len))))
print('Maximum length english sentence: {}'.format(len(max(eng_text_tokenized,key=len))))


# Check language length
french_vocab = len(fra_text_tokenizer.word_index) + 1
english_vocab = len(eng_text_tokenizer.word_index) + 1
print("french vocabulary is of {} unique words".format(french_vocab))
print("English vocabulary is of {} unique words".format(english_vocab))

Maximum length french sentence: 14
Maximum length english sentence: 7
french vocabulary is of 14559 unique words
English vocabulary is of 6828 unique words


In [7]:
max_french_len = int(len(max(fra_text_tokenized,key=len)))
max_english_len = int(len(max(eng_text_tokenized,key=len)))

fra_pad_sentence = pad_sequences(fra_text_tokenized,15, padding = "post")
eng_pad_sentence = pad_sequences(eng_text_tokenized,15, padding = "post")




fra_pad_sentence = fra_pad_sentence.reshape(*fra_pad_sentence.shape, 1)
eng_pad_sentence = eng_pad_sentence.reshape(*eng_pad_sentence.shape, 1)

In [8]:
input_sequence = Input(shape=(15,))
embedding = Embedding(input_dim=french_vocab, output_dim=128,)(input_sequence)

In [ ]:
encoder = LSTM(64, return_sequences=False)(embedding)
r_vec = RepeatVector(15)(encoder)
decoder = LSTM(64, return_sequences=True, dropout=0.2)(r_vec)
logits = TimeDistributed(Dense(english_vocab))(decoder)


In [ ]:
enc_dec_model = Model(input_sequence, Activation('softmax')(logits))
enc_dec_model.compile(loss=sparse_categorical_crossentropy,
              optimizer=Adam(1e-3),
              metrics=['accuracy'])
enc_dec_model.summary()

In [ ]:
BATCH_SIZE = 32
STEPS_PER_EPOCH = len(fra_pad_sentence) / BATCH_SIZE
SAVE_PERIOD = 10


from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint("model/weights1.{epoch:02d}.hdf5", monitor='loss', verbose=1,
    save_best_only=True, mode='auto', save_freq=int(SAVE_PERIOD * STEPS_PER_EPOCH))

In [ ]:
model_results = enc_dec_model.fit(fra_pad_sentence, eng_pad_sentence, batch_size=32, epochs=70,callbacks=[checkpoint])


Epoch 1/70
1875/1875 [==============================] - 207s 109ms/step - loss: 1.5833 - accuracy: 0.7631
Epoch 2/70
1875/1875 [==============================] - 197s 105ms/step - loss: 1.5533 - accuracy: 0.7631
Epoch 3/70
1875/1875 [==============================] - 198s 106ms/step - loss: 1.5129 - accuracy: 0.7670
Epoch 4/70
1875/1875 [==============================] - 197s 105ms/step - loss: 1.3318 - accuracy: 0.7869
Epoch 5/70
1875/1875 [==============================] - 199s 106ms/step - loss: 1.1930 - accuracy: 0.8029
Epoch 6/70
1875/1875 [==============================] - 201s 107ms/step - loss: 1.0910 - accuracy: 0.8154
Epoch 7/70
1875/1875 [==============================] - 198s 105ms/step - loss: 1.0035 - accuracy: 0.8256
Epoch 8/70
1875/1875 [==============================] - 199s 106ms/step - loss: 0.9301 - accuracy: 0.8337
Epoch 9/70
1875/1875 [==============================] - 197s 105ms/step - loss: 0.8663 - accuracy: 0.8405
Epoch 10/70
1875/1875 [=======================

In [ ]:
#enc_dec_model.save('my_model_v2.h5')

In [17]:
import tensorflow as tf
new_enc_dec_model = tf.keras.models.load_model('my_model_v2.h5')



In [ ]:
x=["bonjour à tous"]
y=fra_text_tokenizer.texts_to_sequences(x)
z=pad_sequences(y, 15, padding = "post")

# print(y)
z=np.reshape(z,(1,15,1))

def logits_to_sentence(logits, tokenizer):

    index_to_words = {idx: word for word, idx in tokenizer.word_index.items()}
    index_to_words[0] = '' 

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

index = 8000
# print("The english sentence is: {}".format(english_sentences[index]))
print("The french sentence is: {}".format(x[0]))
print('The predicted sentence is :')
print(logits_to_sentence(new_enc_dec_model.predict(z)[0], eng_text_tokenizer))

# print(fra_pad_sentence[50001:50002].shape)
# print(z.shape)


